## ChemProp2
**Authors:** Abzer Kelminal (abzer.shah@uni-tuebingen.de) <br>
**Edited by:** Daniel Petras (daniel.petras@uni-tuebingen.de) <br>
**Accepted Input file formats:** .txt,.tsv and .csv files <br>
**Outputs:** .csv files, .svg images  <br>
**Dependencies:** ggplot2, dplyr, svglite

### About Input files:

- **Feature_file** is obtained by performing Feature based Molecular Networking on the data using MZmine software.
- **Nw_edge file** has the information of Feature IDs that are similar (not the same) in the columns 'Feature_ID_1' & 'Feature_ID_2'. This file is an output of GNPS
- **Clusterinfo file** is an output of FBMN

<div class="alert alert-block alert-warning">
<b>Setting the working directory and creating a result directory:</b>
</div>

In [ ]:
# setting the current directory as the working directory
Directory <- normalizePath(readline("Enter the path of the folder with input files: "),"/",mustWork=FALSE)
setwd(Directory)

In [ ]:
getwd()

In [ ]:
# install the package if not present

#install.packages('ggplot2')
#install.packages('dplyr')
#install.packages('svglite')

In [ ]:
library('ggplot2')
library('dplyr')
library('tidyr')
library('svglite') # saving ggplots as svg

In [ ]:
# Getting all the files in the folder
dirs <- dir(path=paste(getwd(), sep=""), full.names=TRUE, recursive=TRUE)
folders <- unique(dirname(dirs))
files = list.files(folders, full.names=TRUE)
files_1 <- basename((files))
files_2 <- dirname((files))

In [ ]:
# Creating a Result folder
dir.create(path=paste(files_2[[1]], "_Results", sep=""), showWarnings = TRUE)
fName <-paste(files_2[[1]], "_Results", sep="")

<div class="alert alert-block alert-warning">
<b>Specifying the input files for ChemProp calculation:</b>
</div>

**<font color='red'> In the following line, enter the required file ID numbers separated by commas. For example as: 1,2,3 </font>**

In [ ]:
print(files_1)
input <- as.double(unlist(strsplit(readline("Specify the ID numbers of feature-file, metadata, network edge file:"), split=",")))

In [ ]:
#Gets the extension of each file. Ex:csv
pattern <- c()
for (i in files_1){
  sep_file <- substr(i, nchar(i)-2,nchar(i))
  pattern <- rbind(pattern,sep_file)
}
#pattern

In [ ]:
ft <- read.csv(files_1[input[1]],sep = ifelse(pattern[input[1]]!="csv","\t",","), header=TRUE, row.names = 1,check.names = FALSE) # By applying 'row.names = 1', the 1st column 'ID' becomes the row names
md <-read.csv(files_1[input[2]], sep = ifelse(pattern[input[2]]!="csv","\t",","), header=TRUE, row.names = 1,check.names = FALSE)
nw <-read.csv(files_1[input[3]], sep = ifelse(pattern[input[3]]!="csv","\t",","), header = TRUE,check.names = FALSE)

In [ ]:
#If you have cluster_info file:
if(readline("Do you have Cluster information summary file? Y/N:")=="Y"){
  cl <- as.double(readline("Enter the ID number of cluster info file:"))
  cl <- read.csv(files_1[cl],sep=ifelse(pattern[cl]!="csv","\t",","), header = TRUE,check.names = FALSE)
}

Lets check if the data has been read correclty!!

In [ ]:
head(ft) #head function returns the header (upto first 6 rows)of each files.
dim(ft) #looking at the dimensions(rows & columns) of ft

In [ ]:
head(md)
dim(md)

In [ ]:
if(readline("Is Metadata information given column-wise? Y/N:") == "N"){
  md <- as.data.frame(t(md))
} 

In [ ]:
# To remove the empty columns (with only NAs) present in the files
md <- md[,colSums(is.na(md))< nrow(md)]
dim(md)

In [ ]:
ft<- ft[,order(colnames(ft))] #ordering the ft by its column names
md <-md[order(rownames(md)),] #ordering the md by its row names

In [ ]:
print(colnames(ft))
print(rownames(md))

In [ ]:
# Removing Peak area extensions in the names of feature table and metadata if any present

#colnames(ft <- gsub(' Peak area','',colnames(ft))
#rownames(md) <- gsub(' Peak area','',rownames(md))

In [ ]:
#Getting the extension type of the files 
Ext<-unlist(strsplit(readline('Give the extension of your filetype as same as given in ft and md (Ex:mzML, mzXML):'),split=','))
Ext <- gsub(" ","", Ext)  #remove if there is any space in the given input

new_ft <- ft[,grep(Ext[1],colnames(ft))] #getting only the columns with, for ex: mzml on its name
new_md <- md[grep(Ext[2],rownames(md)),] #getting only the rows with, for ex: mzml on its name

<div class="alert alert-block alert-warning">
<b> Arranging the feature file and metadata:</b>
</div> <br>
In order to perform ChemProp calculation, arrange the feature file and metadata in such a way that the column names of feature table and rownames in metadata table are the same.

In [ ]:
#lists the colnames(ft) which are not present in md
unmatched_ft <- colnames(new_ft)[which(is.na(match(colnames(new_ft),rownames(new_md))))] 
cat("These columns of feature table are not present in metadata:", unmatched_ft)

In [ ]:
#lists the rownames of md which are not present in ft
unmatched_md <- rownames(new_md)[which(is.na(match(rownames(new_md),colnames(new_ft))))] 
cat("These rows of metadata are not present in feature table:", unmatched_md)

In [ ]:
#Removing those unmatching columns and rows:
if(is.null(unmatched_ft)==F){new_ft <- subset(new_ft, select = -c(which(is.na(match(colnames(new_ft),rownames(new_md))))) )}
if(is.null(unmatched_md)==F){new_md <- new_md[-c(which(is.na(match(rownames(new_md),colnames(new_ft))))),]}

In [ ]:
dim(new_ft)
dim(new_md)

In [ ]:
identical(colnames(new_ft),rownames(new_md))#checking if they are the same

Now, we have both our feature table and metadata in the same order.

<div class="alert alert-block alert-warning">
<b> Subsetting the feature table and metadata based on user-defined condition:</b>
</div>

In [ ]:
head(new_md)
print(matrix(data=colnames(new_md),nrow=length(colnames(new_md))))

In [ ]:
Condition <- as.double(unlist(strsplit(readline("Enter the IDs of interested attributes separated by commas:"),split=",")))

In [ ]:
#Specifying which conditions to keep under the selected attributes

Meta_Filtered <-new_md
for(i in 1:length(Condition)){
    
    #Shows the different levels within each selected condition:
    Levels_Cdtn <- levels(as.factor(Meta_Filtered[,Condition[i]]))
    print(matrix(Levels_Cdtn,length(Levels_Cdtn)))
    
    #These lines are not needed in R console, but in Jupyter Notebook to get the previous print statement working
    flush.console()  
    Sys.sleep(0.2)
    
    #Among the shown levels of ana ttribute, select the ones to keep
    Cdtn <- as.double(unlist(strsplit(readline("Enter the IDs of condition(s) you want to KEEP (separated by commas):"), split=',')))
    Levels_Cdtn[Cdtn]
    
    #Selecting only rows in meta_filtered that match the condition
    Meta_Filtered <- Meta_Filtered[(Meta_Filtered[,Condition[i]] == Levels_Cdtn[Cdtn]),]
  }

#Removing all the rows with only NAs
Meta_Final <- subset(Meta_Filtered,rowSums(is.na(Meta_Filtered))!=ncol(Meta_Filtered))

In [ ]:
head(Meta_Final)
dim(Meta_Final)

In [ ]:
#Picking only the columns in ft that are same as rownames in final metadata
ft_Condition <- new_ft[,which(colnames(new_ft)%in%rownames(Meta_Final))] 

In [ ]:
head(ft_Condition)
dim(ft_Condition)

**Filtering the metadata attribute to be used for ChemProp2 correlation calculation:**
- It is important to have the attribute in a numeric form as it is used in calculating ChemProp scores. Hence, any letters, if present in the attribute, should be removed.
- The below lines pick the attribute containing the longitudinal data and does the above mentioned.

In [ ]:
print(matrix(data=colnames(new_md),nrow=length(colnames(new_md))))
md_ID <- as.double(readline("Enter the ID of the Metadata attribute for calculating ChemProp:"))

In [ ]:
Meta_ChemProp <- Meta_Final %>% select(contains(colnames(Meta_Final)[md_ID]))
Meta_ChemProp

print(levels(as.factor(Meta_ChemProp[,1]))) #seeing the levels of the attribute for ChemProp calculation

In [ ]:
if(readline('Are there any letters present in the attribute? Y/N:')=='Y')
{Meta_ChemProp[,1] <- gsub(readline('Enter the letter to be removed from the ChemProp attribute:'),'',Meta_ChemProp[,1])}

Meta_ChemProp

In [ ]:
typeof(Meta_ChemProp[,1])

In [ ]:
Meta_ChemProp[,1] <- as.numeric(Meta_ChemProp[,1])
typeof(Meta_ChemProp[,1])

### Chemical Proportionality score:

- The below code adds a column of **Chemical Proportionality score** to the Nw_edge file. In addition to that, columns with information such as absolute values of ChemProp score and the sign of Chemprop scores are also added.
- In addition to ChemProp score using Pearson correlation method (which is ideal for linear transformations), the below code also generates scores using other methods such as spearman correlation, natural log transformation, square root transformations, for supporting non-linear data 

In [ ]:
Feature_file <- ft_Condition
#Meta_ChemProp[,1] <- as.numeric(Meta_ChemProp[,1])

In [ ]:
head(Feature_file)
#Does the rownames of the Feature table only contains the ID information:
if(readline('Does the rownames of the Feature table only contains the row ID? Y/N:')=='N'){
    Split_Values <- strsplit(as.character(rownames(Feature_file)),'_') 
    Feature_file <- data.frame(do.call(rbind, Split_Values),Feature_file)
    rownames(Feature_file) <- Feature_file[,1]
    Feature_file <- Feature_file[,-1:-3]
    colnames(Feature_file) <- gsub('X','',colnames(Feature_file))
}

In [ ]:
head(Feature_file)

In [ ]:
ChemProp2 <- c()
ChemProp_spearman <-c()
ChemProp_log <- c()
ChemProp_sqrt <- c()

for (i in 1:NROW(Nw_edge)) {
  
  x<- subset(Feature_file, rownames(Feature_file) == Nw_edge[i,1]) # rownames(Feature_file) is the feature ID or cluster ID. The subset command gets the 'Feature ID 1' from the first column of Nw_edge. Then picks the row from the Feature_file corresponding to the 'Feature ID 1'
  x<- rbind(x,subset(Feature_file, rownames(Feature_file) == Nw_edge[i,2]))
  # x is the subset data which has the Feature ID 1 and 2 specified according to Nw_edge file.
  reorder_id<-match(rownames(Meta_ChemProp),colnames(x)) #Match gives the position in which B (the column names of Meta data) is present in A (subset data) and store the position info in reorder_id 
  reordered_x <- data.frame(t(x[reorder_id])) #Rearranging x (subset data) with respect to the new positions and transposing it
  reordered_x <- cbind(Meta_ChemProp[,1],reordered_x) # Combining the metadata column (here, timepoint) with reordered_x
  #Thus, the resulting reordered_x contains 3 columns, such as: 'Metadata info(eg., Timepoint)', 'Feature ID 1', 'Feature ID 2'
  
  corr_result<-cor(reordered_x, method = "pearson") # Performing Pearson correlation
  ChemProp_score <- (corr_result[1,3] - corr_result[1,2]) / 2 # ChemProp2 score is obtained by: (Pearson(Feature ID 2) - Pearson(Feature ID 1)) / 2
  
  corr_2 <- cor(reordered_x, method = "spearman") # Performing Spearman correlation
  Score_spearman <- (corr_2[1,3] - corr_2[1,2]) / 2
  
  log_reorderedX <- cbind(reordered_x[,1],log(reordered_x[,2:3]+1)) # Performing natural log transformations on Feature IDs 1 and 2
  corr_3 <- cor(log_reorderedX) # performing (pearson) correlation on the log transformed data
  Score_log <-(corr_3[1,3] - corr_3[1,2]) / 2
  
  sqrt_reorderedX <- cbind(reordered_x[,1],sqrt(reordered_x[,2:3])) # Taking square roots of Feature IDs 1 and 2
  corr_4 <- cor(sqrt_reorderedX) # performing (pearson) correlation on the square roots
  Score_sqrt <- (corr_4[1,3] - corr_4[1,2])/2
  
  ChemProp2 <- rbind(ChemProp2, ChemProp_score, deparse.level = 0) # deparse.level = 0 constructs no labels; if not given, the resultant matrix has row names (for all rows) created from the input arguments such as 'ChemProp_score' here.
  ChemProp_spearman <- rbind(ChemProp_spearman,Score_spearman,  deparse.level = 0)
  ChemProp_log <- rbind(ChemProp_log,Score_log,  deparse.level = 0)
  ChemProp_sqrt <- rbind(ChemProp_sqrt, Score_sqrt, deparse.level = 0)
}
    
Nw_edge_new <- cbind (Nw_edge, ChemProp2,ChemProp_spearman,ChemProp_log,ChemProp_sqrt)
rownames(Nw_edge_new) <- NULL
#Nw_edge_new <- Nw_edge_new[order(Nw_edge_new$ChemProp2, decreasing = TRUE), ] # Rearranging Nw_edge_new in the decreasing order of ChemProp2 score

Abs_values <- abs(Nw_edge_new[,,(length(Nw_edge)+1):(length(Nw_edge)+4)])
colnames(Abs_values) <- paste("abs", colnames(Abs_values), sep = "_")

Sign_ChemProp2 <- sign(Nw_edge_new$ChemProp2) #getting only the sign of ChemProp2 as 1 or -1
         
ChemProp2_file <- cbind(Nw_edge_new,Abs_values,Sign_ChemProp2)

In [ ]:
head(ChemProp2_file)
#write.csv(ChemProp2_file,paste0(fName,'/20220608_ChemProp2_Result.csv'),row.names = F)

<div class="alert alert-block alert-warning">
<b>Plotting scatterplots to see interesting mass changes and saving them automatically onto result folder:</b>
</div> 

The below condition gets the scatterplots of network pairs with ChemProp2 scores lower than -0.8. Instead of ChemProp2, one can aslo use 'DeltaMZ' to see plots for particular mass changes.  
For ex: `requiredRows <- which(Nw_edge_new$DeltaMZ == -42.011 & is.na(ChemProp2)!=T)`

In [ ]:
requiredRows <- which(Nw_edge_new$DeltaMZ == -42.011 & is.na(ChemProp2)!=T) 
print(paste0("No.of Scatter Plots in the Results Folder will be: ",length(requiredRows)))

In [ ]:
# Function for synthesizing the polynomial regression:
PolyRegression <- function(input_x,input_y, deg){
  poly_model <- lm(input_y ~ poly(input_x,degree=deg))
  pred_res <- predict(poly_model, newdata = data.frame(input_x))
}

In [ ]:
# Specifying a degree for our polynomial fit. One can play around with these values to get the best fit. 
PolyDeg <- as.numeric(readline('Enter a value between 1-10:'))

**Check if you excluded the controls, if not, exclude it** 

In [ ]:
for (i in requiredRows){
  print(i)
  y<- subset(Feature_file, rownames(Feature_file) == Nw_edge[i,1]) # rownames(Feature_file) is the feature ID or cluster ID.
  y<- rbind(y,subset(Feature_file, rownames(Feature_file) == Nw_edge[i,2]))
  reordered_y <- data.frame(t(y[match(rownames(Meta_ChemProp),colnames(y))])) #Rearranging x (subset data) with respect to the new positions and transposing it
  reordered_y <- cbind(Meta_ChemProp[,1],reordered_y) 
  
  reordered_y <- reordered_y[-grep('_C',rownames(reordered_y)),]   #Excluding the controls
  
  par(mar=c(5,4,4,6), mgp=c(2, 1,0), cex.axis=1, cex.lab=1, cex.main=1,xpd=FALSE)
  #svglite(filename=paste0(fName, "/ScatterPlot_RowNo_",i,"_ChemProp_",round(ChemProp_score[i],4), ".svg", sep=""), width=10, height=8, bg='white')
  plot(reordered_y[,1],reordered_y[,2],pch=16,
       main=paste0("Scatter Plot of Feature IDs: ",gsub('X','',colnames(reordered_y[2]))," vs ",gsub('X','',colnames(reordered_y[3]))), 
       sub=paste0("ChemProp2 score: ",round(ChemProp_score,5)),
       col="red",xlab = MetaData_Name,
       ylab= paste0("Abundance: ID ",gsub('X','',colnames(reordered_y[2]))))
  lines(reordered_y[,1],PolyRegression(reordered_y[,1],reordered_y[,2],deg=PolyDeg),col="red")
  
  par(new = TRUE) # Add a new secondary plot
  plot(reordered_y[,1],reordered_y[,3],pch=16, col="blue", # Create second plot without axes
       axes=FALSE, xlab = "", ylab = "")
  ymin = round(min(reordered_y[,3]),-1)
  ymax = round(max(reordered_y[,3]),-1)
  
  lines(reordered_y[,1],PolyRegression(reordered_y[,1],reordered_y[,3],deg=PolyDeg),col="blue")
  
  axis(side = 4, at =round(seq(ymin,ymax,length.out = 5)))   # Add second axis
  mtext( paste0("Abundance: ID ",gsub('X','',colnames(reordered_y[3]))), side = 4,line=2)  
  legend("top",inset=c(0.02,0),
         legend=c(gsub('X','ID ',colnames(reordered_y[2])), gsub('X','ID ',colnames(reordered_y[3]))),
         col=c("red", "blue"), 
         lty=1:2, cex=0.8,pch=16)
    
  #dev.off()

}

<div class="alert alert-block alert-warning">
<b>Combining the information from clusterinfo file onto ChemProp file:</b>
</div> 

In [ ]:
ChemProp_info <- c()
for (i in 1:nrow(ChemProp2_file)){
  y1<- subset(cl, ChemProp2_file[i,1] == cl$`cluster index`)
  colnames(y1) <- paste("Compound1", colnames(y1), sep = "_")
  y2<- subset(cl, ChemProp2_file[i,2] == cl$`cluster index`)
  colnames(y2) <- paste("Compound2", colnames(y2), sep = "_")
  Final <- cbind(ChemProp2_file[i,],y1[,31:ncol(y1)],y2[,31:ncol(y2)])
  ChemProp_info <- rbind(ChemProp_info,Final)
}

ChemProp_NAs_replaced <- ChemProp2_file %>% mutate_if(is.numeric, ~replace(., is.na(.), 0)) # NA values replaced with zeros

write.csv(ChemProp_info,paste0(fName,'/20220608_ChemProp2_Result_AlsoWithClusterInfo.csv'),row.names = F)
write.csv(ChemProp_NAs_replaced,paste0(fName,'/20220608_ChemProp2_replaced_NAs.csv'),row.names=F)

<div class="alert alert-block alert-warning">
<b>Visualizing the distribution of different ChemProp scores of the sample data:</b>
</div>

In [ ]:
bins <- seq(-1,1,0.1)
SCORES<- c()

for (i in (length(Nw_edge)+1):(length(Nw_edge)+4)){
  scores<- cut(as.matrix(ChemProp2_file[,i]),bins,labels=as.character(seq(-0.9,1,0.1))) #cut function store the data into the appropriate bins
  scores_table<-cbind(transform(table(scores)), Condition=paste0("Freq_",names(ChemProp2_file[i])))
  SCORES <- rbind(SCORES,scores_table)
}

HistPlot <- ggplot(SCORES, aes(scores, Freq, fill = Condition)) +
  geom_bar(stat="identity", position = "dodge", width=0.8) + 
  scale_fill_brewer(palette = "Set1") +
  ggtitle(label="Frequency plot") +
  theme(text = element_text(size=14)) +
  xlab("Range") + ylab("Frequency") + labs(fill = "Frequency scores:") + 
  theme(text = element_text(size=12,face="bold"),
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1),   # setting the angle for the x label
        axis.text.y = element_text(angle = 45, vjust = 0.5, hjust=1)) + # setting the angle for the y label
  theme_bw() + #white background and gray grid lines
  theme(plot.title = element_text(hjust = 0.5,size=16,face = "bold"))   # centering the plot title 
  
HistPlot